In [21]:
import sqlite3
con = sqlite3.connect("products.db")
cur = con.cursor()

In [22]:
cur.execute("DROP TABLE IF EXISTS Movies")
cur.execute("DROP TABLE IF EXISTS MovieTheaters")

In [23]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS Movies (
  Code INTEGER,
  Title VARCHAR(255) NOT NULL,
  Rating VARCHAR(255),
  PRIMARY KEY (Code)
);

CREATE TABLE IF NOT EXISTS MovieTheaters (
  Code INTEGER,
  Name VARCHAR(255) NOT NULL,
  Movie INTEGER,
  PRIMARY KEY (Code)
  );

INSERT INTO Movies(Code,Title,Rating) VALUES(1,'Citizen Kane','PG');
INSERT INTO Movies(Code,Title,Rating) VALUES(2,'Singin'' in the Rain','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(3,'The Wizard of Oz','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(4,'The Quiet Man',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(5,'North by Northwest',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(6,'The Last Tango in Paris','NC-17');
INSERT INTO Movies(Code,Title,Rating) VALUES(7,'Some Like it Hot','PG-13');
INSERT INTO Movies(Code,Title,Rating) VALUES(8,'A Night at the Opera',NULL);

INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(1,'Odeon',5);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(2,'Imperial',1);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(3,'Majestic',NULL);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(4,'Royale',6);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(5,'Paraiso',3);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(6,'Nickelodeon',NULL);

""")
con.commit()

In [24]:
# -- 4.1 Select the title of all movies.
cur.execute("SELECT Title FROM Movies")
display(cur.fetchall())

[('Citizen Kane',),
 ("Singin' in the Rain",),
 ('The Wizard of Oz',),
 ('The Quiet Man',),
 ('North by Northwest',),
 ('The Last Tango in Paris',),
 ('Some Like it Hot',),
 ('A Night at the Opera',)]

In [25]:
# -- 4.2 Show all the distinct ratings in the database.
cur.execute("SELECT DISTINCT Rating FROM Movies")
display(cur.fetchall())

[('PG',), ('G',), (None,), ('NC-17',), ('PG-13',)]

In [26]:
# -- 4.3 Show all unrated movies.
cur.execute("SELECT Title FROM Movies WHERE Rating IS NULL")
display(cur.fetchall())

[('The Quiet Man',), ('North by Northwest',), ('A Night at the Opera',)]

In [27]:
# -- 4.4 Select all movie theaters that are not currently showing a movie.
cur.execute("SELECT Name FROM MovieTheaters WHERE Movie IS NULL")
display(cur.fetchall())

[('Majestic',), ('Nickelodeon',)]

In [28]:
# -- 4.5 Select all data from all movie theaters and, additionally, the data from the movie that is being shown in the theater (if one is being shown).
cur.execute("""
    SELECT *
    FROM MovieTheaters
    LEFT JOIN Movies ON MovieTheaters.Movie = Movies.Code
""")
display(cur.fetchall())

[(1, 'Odeon', 5, 5, 'North by Northwest', None),
 (2, 'Imperial', 1, 1, 'Citizen Kane', 'PG'),
 (3, 'Majestic', None, None, None, None),
 (4, 'Royale', 6, 6, 'The Last Tango in Paris', 'NC-17'),
 (5, 'Paraiso', 3, 3, 'The Wizard of Oz', 'G'),
 (6, 'Nickelodeon', None, None, None, None)]

In [29]:
# -- 4.6 Select all data from all movies and, if that movie is being shown in a theater, show the data from the theater.
cur.execute("""
    SELECT *
    FROM Movies
    LEFT JOIN MovieTheaters ON MovieTheaters.Movie = Movies.Code
""")
display(cur.fetchall())

[(1, 'Citizen Kane', 'PG', 2, 'Imperial', 1),
 (2, "Singin' in the Rain", 'G', None, None, None),
 (3, 'The Wizard of Oz', 'G', 5, 'Paraiso', 3),
 (4, 'The Quiet Man', None, None, None, None),
 (5, 'North by Northwest', None, 1, 'Odeon', 5),
 (6, 'The Last Tango in Paris', 'NC-17', 4, 'Royale', 6),
 (7, 'Some Like it Hot', 'PG-13', None, None, None),
 (8, 'A Night at the Opera', None, None, None, None)]

In [30]:
# -- 4.7 Show the titles of movies not currently being shown in any theaters.
cur.execute("""
    SELECT Title
    FROM Movies
    LEFT JOIN MovieTheaters ON MovieTheaters.Movie = Movies.Code
    WHERE Movie IS NULL
""")
cur.fetchall()

[("Singin' in the Rain",),
 ('The Quiet Man',),
 ('Some Like it Hot',),
 ('A Night at the Opera',)]

In [31]:
# -- 4.8 Add the unrated movie "One, Two, Three".
cur.execute("""
    INSERT INTO Movies(Code,Title,Rating)
    VALUES(9,'One, Two, Three',NULL)
""")
con.commit()

In [32]:
# -- 4.9 Set the rating of all unrated movies to "G".
cur.execute("""
    UPDATE Movies
    SET Rating = 'G'
    WHERE Rating IS NULL
""")
con.commit()

In [33]:
# -- 4.10 Remove movie theaters projecting movies rated "NC-17".
cur.execute("""
    DELETE FROM MovieTheaters
    WHERE Code IN (
        SELECT MovieTheaters.Code
        FROM MovieTheaters
        LEFT JOIN Movies ON MovieTheaters.Movie = Movies.Code
        WHERE Movies.Rating = 'NC-17'
    )
""")
con.commit()